# BYTE-PAIR-ENCODING del Quijote
--- 
Este es una forma simple de compresión de datos en la que el par más común de bytes consecutivos de datos se reemplaza con un byte que no ocurre dentro de esos datos. Aquí, el objetivo no es la compresión de datos, sino la codificación de texto en un idioma dado como una secuencia de 'tokens', utilizando un vocabulario fijo de diferentes tokens. La mayoría de las palabras se codificarán como un solo token, mientras que las palabras raras se codificarán como una secuencia de unos pocos tokens, donde estos tokens representan partes de palabras significativas.

### Equipo cangrejo      
* Montaño Preciado Alondra Karolina
* Velasquez Hidalgo Luis Juventino
* Navarro Lopez Malcom Hiram
* Faz Leal Juan Carlos


### Fuentes
- Medium con la informacion: https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0

- Codigo en el cual nos estamos inspirando: https://leimao.github.io/blog/Byte-Pair-Encoding/

In [3]:
import re, collections

In [4]:
def clean_text(text, clean_type='all'):
    if clean_type == 'commas':
        text = re.sub(r'[^\w\s]', '', text)
    elif clean_type == 'numbers':
        text = re.sub(r'\d', '', text)
    else:
        text = re.sub(r'[^\w\s]|\d', '', text)
    return text

In [5]:
print(clean_text("*** START OF THIS PROJECT GUTENBERG EBOOK DON QUIJOTE ***"))
print(clean_text("Posting Date: April 27, 2010 [EBook #2000]"))
print(clean_text("YO, EL REY."))
print(clean_text("-Por Dios, hermano, vuestras aciones."))
print(clean_text("»En lo que toca el poner anotaciones al fin del libro,"))
print(clean_text("''¿Para qué conmigo flo-?''"))

 START OF THIS PROJECT GUTENBERG EBOOK DON QUIJOTE 
Posting Date April   EBook 
YO EL REY
Por Dios hermano vuestras aciones
En lo que toca el poner anotaciones al fin del libro
Para qué conmigo flo


### get_vocab()
___
La función get_vocab lee un archivo de texto (especificado por filename) y devuelve un diccionario de frecuencias de palabras (vocabulario).
Cada línea en el archivo de texto se divide en palabras y cada palabra se agrega al vocabulario con un contador de frecuencia inicial de 1.
Si la palabra ya existe en el vocabulario, su contador se incrementa en 1.

- filename: Es un argumento de entrada para la función, es el nombre (o la ruta) del archivo de texto que se va a leer para crear el vocabulario.
- return: Es una instrucción que se utiliza para devolver un valor desde la funcion, en este caso, la función get_vocab devuelve el diccionario de frecuencias
  de palabras (vocabulario) creado en el cuerpo de la función, que se almacena en la variable vocab.


In [6]:
def get_vocab(filename):
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            line = clean_text(line)
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

### get_stats()
___
La función get_stats toma un vocabulario, el cual tiene la forma de un diccionario, y retorna otro diccionario con los pares de simbolos de nuestro vocabulario y la frecuencia con la que estos pares aparecen. Si el par ya existe en el diccionario, su contador se incrementa en 1.

In [7]:
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### merge_vocab()
___
Funcion que junta los dos tokens que mas aparecen consecutivamente en el vocabulatio.
El bigrama es la union de los dos tokens que mas aparecen. 

Si la palabra contiene el bigrama, "w_out" obtiene el valor de la palabra actual remplazando los dos tokens por el bigrama, de lo contrario "w_out" obtiene conserva el valor de la palabra de 

la instruccion:
`v_out[w_out] = v_in[word]`
nos asegura que la frecuencia de la *posible* nueva palabra se conserva.
- pari: El par de tokens que mas se repiten  
- v_in: El vocabulario actual de palabras.
- return: El nuevo vocabulario, donde la ocurrencia del par de tokens 'pair' se remplazo por su union.

In [8]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')

    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

### get_tokens_from_vocab()
___
Es una función que produce dos objetos a partir de un vocabulario. Uno de los objetos contendra un diccionario que asocia cada símbolo individual en el vocabulario a la suma de las frecuencias de todas las palabras que contienen ese token. El otro objeto también es un diccionario pero este asocia acada palabra del vocabulario una lista de tokens individuales.

como un diccionario predeterminado y luego recorre cada entrada en vocabulario. Para cada entrada, la función divide la palabra en tokens individuales y agrega la frecuencia de la palabra a la frecuencia total de cada token. Además, la función agrega una entrada al diccionario que asocia la palabra completa con su lista de tokens. Finalmente, la función devuelve ambos diccionarios.

In [9]:
def get_tokens_from_vocab(vocab):
    # se crea un diccionario de frecuencias y de tokenizacion de tokens vacio
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    # se recorre el vocabulario de palabras y frecuencias de palabras del corpus
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
        # se devuelve el diccionario de frecuencias de tokens y el diccionario de tokenizacion de tokens
    return tokens_frequencies, vocab_tokenization


### measure_token_length()
___

La función measure_token_lengt mide la longitud de un token de texto. Si el token termina en </w>, se devuelve
la longitud de la palabra sin el sufijo </w> más 1. 
Si el token no termina en </w>, se devuelve la longitud del token
tal como está. Esto significa que la función devuelve la longitud de la palabra real o la longitud de la palabra más el sufijo </w>.

-token: Es un argumento de entrada para la función measure_token_length. Es una cadena de texto que
representa una palabra o una secuencia de caracteres. 

La función mide su longitud y devuelve el resultado.
-return len(token): Es una instrucción que devuelve la longitud de la cadena de texto token. La función len en
python devuelve la cantidad de caracteres en una cadena de texto.
En este caso, si token no termina en </w>, se devuelve su longitud usando return len(token). De lo contrario,
se devuelve len(token[:-4]) + 1, lo que representa la longitud de token sin los últimos 4
caracteres (que son </w>) más 1.

In [10]:
def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

### tokenize_word()
___
Es una función de tokenización de palabras basada en el algoritmo de búsqueda de coincidencia de una lista de tokens conocidos. Toma como entrada una cadena de texto y una lista de tokens conocidos ordenados. Si la cadena de texto y la lista de tokens conocidos no son vacías, se itera sobre la lista de tokens conocidos en orden y se buscan coincidencias con la cadena de texto

In [11]:
### Funcion para tokenizar una palabra con un vocabulario de tokens ordenados por frecuencia y longitud de token ###
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    
    if sorted_tokens == []:
        return [unknown_token]
    string_tokens = []
    # se recorre el vocabulario de tokens ordenados por frecuencia y longitud de token #
    for i in range(len(sorted_tokens)):
    
        token = sorted_tokens[i]
        
        # se reemplaza el punto por [.] para que no sea interpretado como cualquier caracter #
        token_reg = re.escape(token.replace('.', '[.]'))
        
        # se busca el token en la palabra y se obtienen las posiciones de inicio y fin de cada ocurrencia #
        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        
        #si no se encuentra el token en la palabra se continua con el siguiente token #
        if len(matched_positions) == 0:
            continue
            # si se encuentra el token en la palabra se obtienen las posiciones de inicio y fin de cada ocurrencia #
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        # se recorre la palabra desde el inicio hasta la posicion de inicio de cada ocurrencia del token #
        substring_start_position = 0
        
        for substring_end_position in substring_end_positions:
            # se obtiene la subcadena desde la posicion de inicio de la palabra hasta la posicion de inicio de la ocurrencia del token #
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            #se actualiza la posicion de inicio de la palabra para continuar desde la posicion de fin de la ocurrencia del token #
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        # se tokeniza la subcadena restante de la palabra #
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    # se devuelve la lista de tokens de la palabra #
    return string_tokens

## Implementacion usando el Quijote
---


Aqui implementamos el codigo de Byte-pair-encoding

In [12]:
vocab = get_vocab('primer_capitulo.txt')

print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

num_merges = 10
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

# Let's check how tokenization will be for a known word
word_given_known = 'medio</w>'
word_given_unknown = 'Ilikeeatingapples!</w>'

# Ordena el diccionario tokens_frequencies usando la combinacion de dos criterios:
# 1. La llamada a la funcion measure_token_length
# 2. La frecuencia de los tokens.
# Los tokens con una longitud mayor en el resultado de measure_token_length serán
# primero en la lista, y si hay varios tokens con la misma longitud, entonces aquellos
# con una frecuencia más alta serán primero en la lista.
sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
# Es una lista de los tokens ordenados en función de la longitud de los tokens y sus frecuencias
# donde los tokens con mayor longitud y mayor frecuencia se encuentran en la parte superior de la lista.
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

Tokens Before BPE
All tokens: dict_keys(['T', 'h', 'e', '</w>', 'P', 'r', 'o', 'j', 'c', 't', 'G', 'u', 'n', 'b', 'g', 'E', 'B', 'k', 'f', 'D', 'Q', 'i', 'y', 'M', 'l', 'd', 'C', 'v', 'a', 's', 'S', 'w', 'm', 'Y', 'p', 'L', 'A', 'R', 'O', 'F', 'H', 'I', 'J', 'U', 'N', 'K', 'x', 'q', 'á', 'ñ', 'é', 'í', 'V', 'ó', 'ú', 'z', 'É', 'Ó', 'X', 'Z', 'ü', 'Á', 'Í'])
Number of tokens: 63
Iter: 0
Best pair: ('e', '</w>')
All tokens: dict_keys(['T', 'h', 'e</w>', 'P', 'r', 'o', 'j', 'e', 'c', 't', '</w>', 'G', 'u', 'n', 'b', 'g', 'E', 'B', 'k', 'f', 'D', 'Q', 'i', 'y', 'M', 'l', 'd', 'C', 'v', 'a', 's', 'S', 'w', 'm', 'Y', 'p', 'L', 'A', 'R', 'O', 'F', 'H', 'I', 'J', 'U', 'N', 'K', 'x', 'q', 'á', 'ñ', 'é', 'í', 'V', 'ó', 'ú', 'z', 'É', 'Ó', 'X', 'Z', 'ü', 'Á', 'Í'])
Number of tokens: 64
Iter: 1
Best pair: ('o', '</w>')
All tokens: dict_keys(['T', 'h', 'e</w>', 'P', 'r', 'o', 'j', 'e', 'c', 't', '</w>', 'G', 'u', 'n', 'b', 'g', 'E', 'B', 'k', 'f', 'D', 'Q', 'i', 'y', 'M', 'l', 'd', 'C', 'v', 'a', '

# Conclusiones
___
Me diverti mucho